In [1]:
import re
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma


3.10.13 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:24:38) [MSC v.1916 64 bit (AMD64)]


In [ ]:
loader=PyPDFLoader('Yongshun.pdf')
documents=loader.load()
documents = documents[1:]
documents_texts = '\n'.join([doc.page_content[1:] for doc in documents])
documents_texts = re.sub(r'(第[一二三四五六七八九十零百千万]+條)', r'@\1', documents_texts)
documents_texts = re.sub(r'(第[一二三四五六七八九十]+章.*?)(?=\n)', '', documents_texts)
documents_texts = documents_texts.replace('\n', '').replace(' ','')

for i in range(1,16):
    text="永純化學工業股份有限公司工作規則桃園市政府113年1月9日府勞條字第1120369127號函核備文件名稱文件編號版次總頁頁次工作規則YS-A-01C15"+str(i)
    text2="永純化學工業股份有限公司工作規則文件名稱文件編號版次總頁頁次工作規則YS-A-01C15"+str(i)
    documents_texts = documents_texts.replace(text,"").replace(text2,"")
documents_texts = documents_texts.replace('\n', '').replace(' ','').replace('@', ' ')

text_splitter = RecursiveCharacterTextSplitter(chunk_size=570, chunk_overlap=300)
texts = text_splitter.split_text(documents_texts)
print(documents_texts)

 第一條：（訂立目的）永純化學工業股份有限公司(以下簡稱本公司)為明確規定勞資雙方之權利義務，健全現代經營管理制度，促使勞雇雙方同心協力，並謀事業發展，特依勞動基準法及相關法令訂定本規則。 第二條：（適用範圍）凡受本公司僱用從事工作獲致工資之員工均適用之，並一體適用於本公司各廠區。(公司：台北市松江路168號11樓；工廠：桃園市蘆竹區坑口里海山路681號) 第三條:（報到手續）新進員工於接到通知後，應依規定之到職日至本公司辦理報到手續，逾期視為自動放棄，該通知因而失其效力，報到時應繳驗下列文件：一、報到通知書。二、本公司所定員工資料卡。三、最高學歷證書及國民身分證正本（核對後發還）。四、公立醫院或全民健康保險特約醫療院所之體格檢查表乙份。五、本人最近半身脫帽正面二寸照片乙張。 第四條：（勞動契約）本公司因業務需要，僱用員工時，得與員工簽訂定期契約或不定期契約。前項定期契約及不定期契約，依勞動基準法相關規定認定之。 第五條：（工作年資計算）員工工作年資之採計方式規定如下：一、定期契約屆滿後或不定期契約因故停止履行後，未滿三個月而訂定新約或繼續履行原約時，員工前後工作年資，應合併計算。二、員工工作年資以服務本公司為限，並自受僱當日起算。適用勞動基準法前已在本公司工作之年資合併計算。三、受本公司調動之工作年資，其年資由本公司續予承認，並應予合併計算。 第六條：（新進試用）本公司得與新進員工約定試用，試用期間九十天，但具特殊技能、專長、經歷，經專案簽准者，不在此限。考核成績不合格者，須終止契約時，依 第七條、 第八條、 第九條、 第十條及 第十一條等相關規定辦理，工資發放至停止僱用日為止。 第七條：（經預告終止勞動契約）非有下列情事之一者，本公司不得預告員工終止勞動契約：一、歇業或轉讓時。二、虧損或業務緊縮時。三、不可抗拒力暫停工作在一個月以上時。四、業務性質變更，有減少員工之必要，又無適當工作可供安置時。五、員工對於所擔任之工作確不能勝任時。 第八條：（終止契約限制期間之例外）員工在產假期間或職業災害醫療期間，本公司不得終止契約。但若本公司遭天災、事變或其他不可抗力致事業不能繼續者，得報主管機關核定後終止勞動契約。 第九條：（資遣預告）依 第七條或前條但書規定終止勞動契約時，預告期間如下：一、繼續工作三個月以上一年未滿者，於十日前預告之。二、繼續工作一年以上三年未滿者，於

In [3]:
embeddings=HuggingFaceEmbeddings(model_name="paraphrase-multilingual-MiniLM-L12-v2")

c:\Users\Public\Anaconda3\envs\breeze2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
if 'db2' in locals(): db2.delete_collection()
db2=Chroma.from_texts(texts,embeddings,persist_directory="db2", collection_metadata={"hnsw:space": "cosine"})

In [11]:
import torch
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model = AutoModelForCausalLM.from_pretrained(
    "MediaTek-Research/Breeze-7B-Instruct-v0_1",
    device_map="auto",
    torch_dtype=torch.bfloat16,
    attn_implementation="flash_attention_2" # optional
)
tokenizer = AutoTokenizer.from_pretrained("MediaTek-Research/Breeze-7B-Instruct-v0_1")

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
llm = HuggingFacePipeline(pipeline=pipe)

Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.20s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=600)
llm = HuggingFacePipeline(pipeline=pipe)

In [16]:
retriever = db2.as_retriever( search_kwargs={'k': 10})

In [17]:
a = retriever.invoke("員工報到時需要繳交哪些文件？")
for i in a:
  print(re.findall(r'第.{1,5}條(?:之.)*', i.page_content))
  print(i.page_content)
  print('='*30)

['第三條', '第四條', '第五條', '第六條', '第七條', '第八條', '第九條', '第十條', '第十一條']
第三條:（報到手續）新進員工於接到通知後，應依規定之到職日至本公司辦理報到手續，逾期視為自動放棄，該通知因而失其效力，報到時應繳驗下列文件：一、報到通知書。二、本公司所定員工資料卡。三、最高學歷證書及國民身分證正本（核對後發還）。四、公立醫院或全民健康保險特約醫療院所之體格檢查表乙份。五、本人最近半身脫帽正面二寸照片乙張。 第四條：（勞動契約）本公司因業務需要，僱用員工時，得與員工簽訂定期契約或不定期契約。前項定期契約及不定期契約，依勞動基準法相關規定認定之。 第五條：（工作年資計算）員工工作年資之採計方式規定如下：一、定期契約屆滿後或不定期契約因故停止履行後，未滿三個月而訂定新約或繼續履行原約時，員工前後工作年資，應合併計算。二、員工工作年資以服務本公司為限，並自受僱當日起算。適用勞動基準法前已在本公司工作之年資合併計算。三、受本公司調動之工作年資，其年資由本公司續予承認，並應予合併計算。 第六條：（新進試用）本公司得與新進員工約定試用，試用期間九十天，但具特殊技能、專長、經歷，經專案簽准者，不在此限。考核成績不合格者，須終止契約時，依 第七條、 第八條、 第九條、 第十條及 第十一條等相關規定辦理，工資發放至停止僱用日為止。
['第三條', '第四條', '第五條', '第六條', '第七條', '第八條', '第九條', '第十條', '第十一條']
第三條:（報到手續）新進員工於接到通知後，應依規定之到職日至本公司辦理報到手續，逾期視為自動放棄，該通知因而失其效力，報到時應繳驗下列文件：一、報到通知書。二、本公司所定員工資料卡。三、最高學歷證書及國民身分證正本（核對後發還）。四、公立醫院或全民健康保險特約醫療院所之體格檢查表乙份。五、本人最近半身脫帽正面二寸照片乙張。 第四條：（勞動契約）本公司因業務需要，僱用員工時，得與員工簽訂定期契約或不定期契約。前項定期契約及不定期契約，依勞動基準法相關規定認定之。 第五條：（工作年資計算）員工工作年資之採計方式規定如下：一、定期契約屆滿後或不定期契約因故停止履行後，未滿三個月而訂定新約或繼續履行原約時，員工前後工作年資，應合併計算。二、員工工作年資以服務本公司為限，並自受僱當日起算。適用勞

In [ ]:
import pandas as pd
from langchain_core.prompts import PromptTemplate
template = """你是問答任務的助手。注意:請根據以下文章來回答問題。注意:如果你在文章中找不到答案，請回答"沒有提及"。答案只能從文章內尋找，請勿根據其他資訊來回答。

文章:{context}

問題:{question}

請詳細回答並說明。

回答:"""
custom_rag_prompt = PromptTemplate.from_template(template)


from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


def format_docs(docs):
    return "".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | custom_rag_prompt
    | llm
    | StrOutputParser()
)

df = pd.read_excel("Questions_Answers.xlsx")
question = df['Query']
answer = df['標準答案_永純化學']
print("模型答案",rag_chain.invoke("假期總類有哪些?"))
for (ques,ans) in zip(question,answer):
  print(ques)
  if ques == None:
    break
  print("模型答案:",rag_chain.invoke(ques))
  print()
  print("標準答案:",ans)
  print("="*50)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


模型答案 假期總類包括例假、休息日、休假日、特別休假、婚假、喪假、公傷病假、產假、產檢假、家庭照顧假、陪產檢及陪產假、安胎休養請假、公假、年假、特別休假(免)、生理假、及法定節假日等。
